In [7]:
from datasets.severstal_coco import DatasetCOCO
from torchvision import transforms
from models.clipseg import CLIPDensePredT
import torch
from PIL import Image
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

In [8]:
data_path = '/home/eas/Enol/pycharm_projects/clipseg_steel_defect/Severstal/train_subimages'
weights = '/home/eas/Enol/pycharm_projects/clipseg_steel_defect/logs/rd64-7K-vit16-cbh-coco-enol-5classes_no_neg/weights.pth'
mean = [0.34388125, 0.34388125, 0.34388125]
std = [0.13965334, 0.13965334, 0.13965334]
image_size = 256
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])
COCO_CLASSES = {1: 'network of fine, hairline cracks or fissures on the surface of the steel',
                2: 'scale—oxides embedded into the steel plate',
                3: 'shallow, narrow grooves or lines on the surface of the steel',
                4: 'impurity or foreign material embedded within the steel matrix',
                5: 'defects on a steel plate'}
model = CLIPDensePredT(version='ViT-B/16', reduce_dim=64)
model.load_state_dict(torch.load(weights, weights_only=True, map_location=torch.device('cpu')), strict=False)
model.cuda()
split = 'train'

In [9]:
df = pd.DataFrame(columns = ['class_id', 'embedding'])

In [10]:
coco_dataset = DatasetCOCO(data_path, transform, split, True)

loading annotations into memory...
Done (t=0.43s)
creating index...
index created!


In [11]:
class_ids = coco_dataset.ids_by_class
metadata = coco_dataset.img_metadata

In [12]:
idx = 0
for i in class_ids:
    class_id = i
    for ann in tqdm(class_ids[class_id]):
        query = metadata.loadAnns(ids=ann)[0]
        image = Image.open(os.path.join(data_path, query['image_id']))
        q = transform(image).unsqueeze(0)
        q = q.cuda()
        visual_q, _, _ = model.visual_forward(q)
        df.loc[idx] = [class_id, visual_q.cpu().detach().numpy()]
        idx += 1


 17%|█▋        | 8406/48985 [02:24<11:39, 58.03it/s]


KeyboardInterrupt: 